# Util Bloqs

The utility bloqs let you reshape Soquets.

In [ ]:
from qualtran import QAny
from qualtran.drawing import GraphDrawer, PrettyGraphDrawer, show_bloq
from qualtran.bloqs.util_bloqs import Split, Join, Partition
import numpy as np

from IPython.display import SVG

## Split and Join

Split and join take a `bitsize=n, shape=()` to `bitsize=1, shape=(n,)` and back.

In [ ]:
show_bloq(Split(QAny(3)))

In [ ]:
show_bloq(Join(QAny(3)))

## SplitJoin

For fun, we can pair `Split` and `Join` into an identity operation.

In [ ]:
from attrs import frozen
from functools import cached_property
from typing import *
from qualtran import Bloq, Signature, Register
from qualtran.bloqs.basic_gates import CNOT

@frozen
class SplitJoin(Bloq):
    n: int

    @cached_property
    def signature(self) -> Signature:
        return Signature([Register('x', QAny(self.n))])

    def build_composite_bloq(
        self, bb: 'BloqBuilder', *, x: 'Soquet'
    ) -> Dict[str, 'Soquet']:
        xs = bb.split(x)
        xs[0], xs[1] = bb.add(CNOT(), ctrl=xs[0], target=xs[1])
        x = bb.join(xs)
        return {'x': x}

In [ ]:
show_bloq(SplitJoin(10))

In [ ]:
show_bloq(SplitJoin(10).decompose_bloq())

## Partition

A Partition is a useful bloq for abstracting away the details of large registers (like multiple selection registers, spin, orbital, ...). We can use a partition bloq to hide these detailed registers until we decompose the bloq further.

In [ ]:
regs = (Register('xx', QAny(2), shape=(2,3)), Register('yy', QAny(37)))
bitsize = sum(reg.total_bits() for reg in regs)
bloq = Partition(n=bitsize, regs=regs)
show_bloq(bloq)

An example of using a `Partition` as part of a decomposition is given below:

In [ ]:
from qualtran import BloqBuilder, Soquet, SoquetT
from qualtran.bloqs.for_testing import TestMultiRegister

@frozen
class BlackBoxBloq(Bloq):
    subbloq: Bloq

    @cached_property
    def bitsize(self):
        return sum(reg.total_bits() for reg in self.subbloq.signature)

    @cached_property
    def signature(self) -> Signature:
        return Signature.build(system=self.bitsize)

    def build_composite_bloq(self, bb: 'BloqBuilder', system: 'SoquetT') -> Dict[str, 'Soquet']:
        bloq_regs = self.subbloq.signature
        partition = Partition(self.bitsize, bloq_regs)
        partitioned_vars = bb.add(partition, x=system)
        partitioned_vars = bb.add(
            self.subbloq, **{reg.name: sp for reg, sp in zip(bloq_regs, partitioned_vars)}
        )
        system = bb.add(
            partition.adjoint(), **{reg.name: sp for reg, sp in zip(bloq_regs, partitioned_vars)}
        )
        return {'system': system}
    
    def short_name(self) -> str:
        return "BBBloq" 

As an example, we'll use the generic `TestMultiRegister` bloq as our sub-bloq with many registers. It does different (contrived) things to the `xx`, `yy`, and `zz` registers:

In [ ]:
subbloq = TestMultiRegister()
show_bloq(subbloq.decompose_bloq())

By wrapping it in `BlackBoxBloq`, the previously-complicated signature is now just one register named "system" with a larger bitsize.

In [ ]:
show_bloq(BlackBoxBloq(subbloq))

`Partition` adapts between the two register sets. We can inspect this in the decomposition:

In [ ]:
show_bloq(BlackBoxBloq(subbloq).decompose_bloq())

## Cast

Split, Joins and Partitions technically cast registers from one QDType to another. E.g. we can split an 32 QBit QInt register into an array of 32 QBits, and then join can be used to reinterpret this as a a different QDType (say a 32 bit QFxp). The Cast bloq cuts out the intermediate splits and joins and simply reinterprets n qubit registers.

In [ ]:
from qualtran import QInt, QFxp
from qualtran.bloqs.util_bloqs import Cast

show_bloq(Cast(QInt(32), QFxp(32, 32)), type='dtype')

This bloq is helpful if we want to explicitly cast registers from one type to another, which may be useful during a complex decomposition involving numeric registers.

In [ ]:
from qualtran.bloqs.for_testing import TestCastToFrom
show_bloq(TestCastToFrom().decompose_bloq(), type='dtype')